In [ ]:
import ollama
import threading

# Initialize the Ollama client
client = ollama.Client()

# Define your batch of prompts
prompts = [
    "What is the capital of France?",
    "Explain the theory of relativity.",
    "How does photosynthesis work?"
]

# Function to process each prompt concurrently
def process_prompt(prompt):
    # Create a message in the format expected by Ollama
    message = {'role': 'user', 'content': prompt}
    # Send the message to the model with streaming enabled
    stream = client.chat(model='deepseek-r1:14b-qwen-distill-q8_0', messages=[message], stream=True)
    
    print(f"\nPrompt: {prompt}")
    # Process each token as it's generated
    for chunk in stream:
        token = chunk['message']['content']
        if prompt == "Explain the theory of relativity.":
            print(token, end='', flush=True)
    print("\n" + "="*50 + "\n")  # Separator between responses

# Create and start a thread for each prompt
threads = []
for prompt in prompts:
    t = threading.Thread(target=process_prompt, args=(prompt,))
    t.start()
    threads.append(t)

# Wait for all threads to finish
for t in threads:
    t.join()


Exception in thread Exception in threading.excepthook:
Exception ignored in thread started by: <bound method Thread._bootstrap of <Thread(Thread-40 (process_prompt), stopped 138638767670976)>>
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1032, in _bootstrap
    self._bootstrap_inner()
  File "/usr/lib/python3.12/threading.py", line 1077, in _bootstrap_inner
    self._invoke_excepthook(self)
  File "/usr/lib/python3.12/threading.py", line 1391, in invoke_excepthook
    local_print("Exception in threading.excepthook:",
  File "/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/ipykernel/iostream.py", line 604, in flush
    self.pub_thread.schedule(self._flush)
  File "/home/mhammed/Desktop/tech_projects/Verbal_Communication_Skills_Trainer/Verbal_Communication_Skills_Trainer_venv/lib/python3.12/site-packages/ipykernel/iostream.py", line 267, in schedule
    self

In [ ]:
import os
import ollama
import subprocess
import json
import time

def load_config(file_path: str):
    """
    Loads environment-variable-like keys from a JSON file and sets them.
    """
    with open(file_path, 'r') as f:
        config_data = json.load(f)
    for key, value in config_data.items():
        # Ensure everything is a string in the environment.
        os.environ[key] = str(value)


def load_model(model_name: str) -> ollama.Client:
    """
    Instantiates an Ollama client for a given model.
    Because environment variables are already set,
    Ollama respects those concurrency/queue settings.
    """
    subprocess.run(["ollama", "pull", model_name], check=True)
    return ollama.Client()

class ClientSingleton:
    _client = None  # Class-level variable to store the singleton instance

    @classmethod
    def get_client(cls, model_name: str) -> ollama.Client:
        """
        Returns the already instantiated client if it exists,
        otherwise creates one using the load_model function.
        """
        if cls._client is None:
            cls._client = load_model(model_name)
        return cls._client
    
# 1) Load config file to set environment variables
load_config("ollama_config.json")

# 2) Create and load your model *once*, after config is applied
# client = ClientSingleton.get_client("deepseek-r1:14b-qwen-distill-q8_0")
client = ClientSingleton.get_client("deepseek-r1:14b-qwen-distill-q4_K_M")

# Function to process each prompt concurrently
def process_prompt(prompt):
    # Create a message in the format expected by Ollama
    message = {'role': 'user', 'content': prompt}
    # Send the message to the model with streaming enabled
    # stream = client.chat(model='deepseek-r1:14b-qwen-distill-q8_0', messages=[message], stream=True)
    stream = client.chat(model='deepseek-r1:14b-qwen-distill-q4_K_M', messages=[message], stream=True)
    
    print(f"\nPrompt: {prompt}")
    # Process each token as it's generated
    for chunk in stream:
        token = chunk['message']['content']
        if prompt == "Tell me a brief story about the moon":
            print(token, end='', flush=True)
    print("\n" + "="*50 + "\n")  # Separator between responses

# Define your batch of prompts
prompts = [
    "What is the capital of France?",
    "Explain the theory of relativity.",
    "How does photosynthesis work?",
    "Tell me a brief story about courage",
    "Tell me a brief story about wisdom",
    "Tell me a brief story about humor",
    "Tell me a brief story about humans",
    "Tell me a brief story about the moon"
]

# Create and start a thread for each prompt
threads = []
for prompt in prompts:
    t = threading.Thread(target=process_prompt, args=(prompt,))
    t.start()
    threads.append(t)

# # A shared data structure: one list per prompt to store tokens
# outputs = [[] for _ in prompts]
# lock = threading.Lock()

# def process_prompt(prompt, idx):
#     """
#     Process a prompt: stream tokens from the model and append each token to outputs.
#     """
#     message = {'role': 'user', 'content': prompt}
#     # Stream tokens from the model
#     stream = client.chat(model='deepseek-r1:14b-qwen-distill-q8_0', messages=[message], stream=True)
#     for chunk in stream:
#         token = chunk['message']['content']
#         with lock:
#             outputs[idx].append(token)
#         # (Optional) small delay to simulate processing time
#         time.sleep(0.05)
#     # Mark the prompt as finished
#     with lock:
#         outputs[idx].append("[DONE]")

# def print_grid():
#     """
#     Repeatedly clears the screen and prints a grid where each column represents a prompt and each row a token step.
#     """
#     # Set a fixed column width (adjust as needed)
#     col_width = 40
#     while True:
#         # Clear the screen (works in many ANSI terminals)
#         print("\033c", end="")  # ANSI escape code to clear screen

#         # Print a header with the prompt names (truncated if too long)
#         header = ""
#         for prompt in prompts:
#             header += prompt[:col_width-2].ljust(col_width)
#         print(header)
#         print("-" * (col_width * len(prompts)))

#         with lock:
#             # Determine the maximum number of token steps among all prompts
#             max_rows = max(len(col) for col in outputs)
#             # Build the grid row by row
#             for row in range(max_rows):
#                 line_parts = []
#                 for col in range(len(outputs)):
#                     # If a token exists for this row in this column, use it; else, use an empty string.
#                     token = outputs[col][row] if row < len(outputs[col]) else ""
#                     line_parts.append(token.ljust(col_width))
#                 print("".join(line_parts))
#             # Check if all prompts have finished by looking for the "[DONE]" marker in each output list.
#             all_done = all(col and col[-1] == "[DONE]" for col in outputs)
#         # Pause briefly before reprinting the grid
#         time.sleep(0.2)
#         if all_done:
#             break

# # Start a thread for each prompt to process tokens concurrently
# threads = []
# for i, prompt in enumerate(prompts):
#     t = threading.Thread(target=process_prompt, args=(prompt, i))
#     t.start()
#     threads.append(t)

# # Continuously update the grid display in the main thread
# print_grid()

# # Wait for all threads to finish (they should be finished once "[DONE]" is printed for each)
# for t in threads:
#     t.join()

# print("\nAll prompts finished.")

# # Wait for all threads to finish
# for t in threads:
#     t.join()


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ 


Prompt: What is the capital of France?

Prompt: Explain the theory of relativity.

Prompt: How does photosynthesis work?

Prompt: Tell me a brief story about courage

Prompt: Tell me a brief story about wisdom

Prompt: Tell me a brief story about humor

Prompt: Tell me a brief story about humans

Prompt: Tell me a brief story about the moon


pulling manifest ⠼ pulling manifest 
pulling 6e9f90f02bb3... 100% ▕████████████████▏ 9.0 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                         
pulling 6e4c38e1172f... 100% ▕████████████████▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕████████████████▏  148 B                         
pulling 3c24b0c80794... 100% ▕████████████████▏  488 B                         
verifying sha256 digest 
writing manifest 
success 






<think>
Alright, so I need to create a brief story about the Moon. Hmm, where do I even start? Well, let's think about what I know about the Moon. It's Earth's natural satellite

, right? It orbits our planet and has been a source of wonder for ages. People have always looked up at it and wondered about its origins and secrets.

Maybe I can personify the Moon to make the story more engaging. Personifying objects or celestial bodies often makes stories more relatable. Let me think about what emotions or characteristics the Moon might have. It's often associated with peace, mystery, and sometimes even loneliness because it's up there all alone in the dark sky.

So, perhaps the story could follow the Moon as a character who feels isolated but finds purpose by illuminating Earth at night. That gives it a sense of mission and connection to humanity. The idea that without the Moon, nights would be dark and people couldn't see as clearly might tie into themes of guidance or companionship.

In [ ]:
import ollama
import threading
import time

# Initialize the Ollama client
client = ollama.Client()

# Define your batch of prompts
prompts = [
    "What is the capital of France?",
    "Explain the theory of relativity.",
    "How does photosynthesis work?"
]

# A shared data structure: one list per prompt to store tokens
outputs = [[] for _ in prompts]
lock = threading.Lock()

def process_prompt(prompt, idx):
    """
    Process a prompt: stream tokens from the model and append each token to outputs.
    """
    message = {'role': 'user', 'content': prompt}
    # Stream tokens from the model
    stream = client.chat(model='deepseek-r1:14b-qwen-distill-q8_0', messages=[message], stream=True)
    for chunk in stream:
        token = chunk['message']['content']
        with lock:
            outputs[idx].append(token)
        # (Optional) small delay to simulate processing time
        time.sleep(0.05)
    # Mark the prompt as finished
    with lock:
        outputs[idx].append("[DONE]")

def print_grid():
    """
    Repeatedly clears the screen and prints a grid where each column represents a prompt and each row a token step.
    """
    # Set a fixed column width (adjust as needed)
    col_width = 40
    while True:
        # Clear the screen (works in many ANSI terminals)
        print("\033c", end="")  # ANSI escape code to clear screen

        # Print a header with the prompt names (truncated if too long)
        header = ""
        for prompt in prompts:
            header += prompt[:col_width-2].ljust(col_width)
        print(header)
        print("-" * (col_width * len(prompts)))

        with lock:
            # Determine the maximum number of token steps among all prompts
            max_rows = max(len(col) for col in outputs)
            # Build the grid row by row
            for row in range(max_rows):
                line_parts = []
                for col in range(len(outputs)):
                    # If a token exists for this row in this column, use it; else, use an empty string.
                    token = outputs[col][row] if row < len(outputs[col]) else ""
                    line_parts.append(token.ljust(col_width))
                print("".join(line_parts))
            # Check if all prompts have finished by looking for the "[DONE]" marker in each output list.
            all_done = all(col and col[-1] == "[DONE]" for col in outputs)
        # Pause briefly before reprinting the grid
        time.sleep(0.2)
        if all_done:
            break

# Start a thread for each prompt to process tokens concurrently
threads = []
for i, prompt in enumerate(prompts):
    t = threading.Thread(target=process_prompt, args=(prompt, i))
    t.start()
    threads.append(t)

# Continuously update the grid display in the main thread
print_grid()

# Wait for all threads to finish (they should be finished once "[DONE]" is printed for each)
for t in threads:
    t.join()

print("\nAll prompts finished.")


In [ ]:
from functools import lru_cache
import numpy as np

def quantize_embedding(embedding: np.ndarray, bits: int = 8, group_size: int = 32):
    """Quantize an embedding using grouped min-max scaling."""
    assert bits in (4, 8)
    max_val = (1 << bits) - 1  # 255 for 8-bit, 15 for 4-bit
    embedding = np.array(embedding, dtype=float)
    quantized = [len(embedding)]
    # Process in groups of group_size
    for i in range(0, len(embedding), group_size):
        chunk = embedding[i : i + group_size]
        m, M = float(chunk.min()), float(chunk.max())
        # Scale values of this chunk to [0, max_val]
        if M == m:
            q_vals = [0] * len(chunk)  # all values equal -> all zeros after quantization
        else:
            q_vals = [int(round(x)) for x in ((chunk - m) / (M - m) * max_val)]
        # Store min, max, and quantized values for this chunk
        quantized.append(m); quantized.append(M)
        quantized.extend(q_vals)
    return tuple(quantized)  # tuple is hashable and can be used as cache key

@lru_cache(maxsize=1000)
def _cached_llm_inference(qtuple: tuple, bits: int = 8, group_size: int = 32):
    """Cached LLM inference: returns output for a given quantized embedding."""
    # **Simulate** an expensive LLM call (here we just return a string result)
    return f"LLM output"

def get_llm_output(embedding: np.ndarray, bits: int = 8, group_size: int = 32):
    """Return LLM output for embedding, using cache to avoid repeat inference."""
    qtuple = quantize_embedding(embedding, bits, group_size)
    return _cached_llm_inference(qtuple, bits, group_size)

# Example usage:
emb = np.random.rand(128)
print(get_llm_output(emb, bits=8))  # First call (cache miss triggers inference)
print(get_llm_output(emb, bits=8))  # Second call (cache hit, returns cached result)



In [ ]:
from cachetools import cached
from cachetools.keys import hashkey

from random import randint

@cached(cache={}, key=lambda db_handle, query: hashkey(query))
def find_object(db_handle, query):
    print("processing {0}".format(query))
    return query

queries = list(range(5))
queries.extend(range(5))
for q in queries:
    print("result: {0}".format(find_object(randint(0, 1000), q)))

In [1]:
from collections import OrderedDict
cache = OrderedDict()

In [3]:
import os
import json
import requests

class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class embedding_wrap(metaclass=Singleton):
    def __init__(self, config_file="ollama_config.json"):
        self.loaded = False
        self.config_data = self.load_config(config_file)
        print(self.config_data)
        response = self.load_model(self.config_data["EMBEDDING_MODEL"])
        if response.status_code == 200: self.loaded = True

    def load_config(self, config_file):
        """
        Loads environment-variable-like keys from a JSON file and sets them.
        """
        with open(config_file, 'r') as f:
            config_data = json.load(f)
        for key, value in config_data.items():
            # Ensure everything is a string in the environment.
            os.environ[key] = str(value)
        return config_data

    def load_model(self, model_name: str):
        """
        Instantiates an Ollama client for a given model.
        Because environment variables are already set,
        Ollama respects those concurrency/queue settings.
        """
        url = self.config_data["LOAD_MODEL_API_PATH"]
        payload = {
            "model": model_name,
            "stream": False  # Return a single response instead of a streamed response
        }

        response = requests.post(url, json=payload)
        print("Status code:", response.status_code)
        print("Response:", response.text)

        return response

    # def get_client(self, model_name: str) -> ollama.Client:
    #     """
    #     Returns the already instantiated client if it exists,
    #     otherwise creates one using the load_model function.
    #     Implements a simple singleton for the client.
    #     """
    #     if self.client is None:
    #         self.client = self.load_model(model_name)
    #     return self.client

    def process_input(self, input: str):
        """
        Sends a prompt to the provided Ollama client and prints the 
        streamed output in real-time.
        """
        # client = self.get_client(model_name)
        
        # message = {'role': 'user', 'content': prompt}
        if self.loaded:
            url = self.config_data["EMBEDDING_API_PATH"]
            payload = {
                        "model": self.config_data["EMBEDDING_MODEL"],
                        "input": input,
                        "keep_alive": "10m",
                        "options": {
                                    "temperature": self.config_data["EMBED_TEMPERATURE"]
                                    }
                        }
            response = requests.post(url, json=payload, stream=False)
            if response.status_code == 200:
                response_data = response.json()
            else:
                return False
            
            # print(response_data, type(response_data))

            return response_data["embeddings"]
            # print(f"\nPrompt: {prompt}")
            # for chunk in stream:
            #     token = chunk['message']['content']
            #     print(token, end='', flush=True)
            # print("\n" + "="*50 + "\n")
        else:
            print("Failed to load the model")


    def run(self, prompt: str, model_name: str = "deepseek-r1:14b-qwen-distill-q4_K_M"):
        """
        Runs the complete workflow: loads config, ensures the model is loaded,
        and processes the prompt.
        """
        return self.process_input(prompt)


# import numpy as np
# embedding = embedding_wrap().run(prompt)
# embedding = np.array(embedding) if embedding else embedding
# print(embedding)


from llm_wrapper import llm_wrap
from embedding_wrapper import embedding_wrap
import numpy as np
import functools

class handle_prompt:
    def __init__(self, prompt, chat = True):
        self.prompt = prompt
        self.chat = chat
        self.llm = llm_wrap()

        self.embedding = embedding_wrap().run(prompt)
        self.embedding = np.array(self.embedding) if self.embedding else self.embedding
        # print(self.embedding.shape)
        # self.embedding = np.array(embedding_wrap().run(prompt))

        self.llm_output = self.get_llm_output()

    def get_llm_output(self, bits: int = 8, group_size: int = 32):
        """Return LLM output for embedding, using cache to avoid repeat inference."""
        if not isinstance(self.embedding, np.ndarray): return self.llm.process_prompt(self.prompt, chat = self.chat)
        embedding = self.quantize_embedding(self.embedding)
        # return _cached_llm_inference(self.prompt, self.llm, qtuple, bits, group_size)
        return _cached_llm_inference(self.prompt, self.llm, self.chat, embedding)
    
    def quantize_embedding(self, embedding: np.ndarray, n_levels: int = 256, min_val: float = None, max_val: float = None) -> np.ndarray:
        # Determine the quantization range.
        if min_val is None:
            min_val = np.min(embedding)
        if max_val is None:
            max_val = np.max(embedding)
        
        # Avoid division by zero for constant embeddings.
        if min_val == max_val:
            return np.zeros_like(embedding, dtype=np.uint8)
        
        # Normalize the embedding to the [0, 1] range.
        normalized = (embedding - min_val) / (max_val - min_val)
        
        # Scale and round to the desired number of quantization levels.
        quantized_embedding = np.round(normalized * (n_levels - 1)).astype(np.uint8)[0].tolist()#[0]
        # print(quantized_embedding)
        
        return tuple(quantized_embedding)

    # # Example usage:
    # # emb = np.random.rand(128)
    # print(get_llm_output(self.embedding, bits=8))  # First call (cache miss triggers inference)
    # print(get_llm_output(self.embedding, bits=8))  # Second call (cache hit, returns cached result)


def lru_cache_by_key(key_func, maxsize=128):
    """
    Decorator that mimics lru_cache but uses a custom key function.
    The key_func accepts the same arguments as the decorated function
    and returns a hashable cache key.
    """
    
    def decorating_function(user_function):

        @functools.wraps(user_function)
        def wrapper(*args, **kwargs):
            key = key_func(*args, **kwargs)
            print("yo key: ",key)
            print(cache)
            if key in cache:
                print("key in it")
                cache.move_to_end(key)  # Mark as recently used
                return cache[key]

            result = user_function(*args, **kwargs)
            print("yo result: ",result)
            cache[key] = result

            if len(cache) > maxsize:
                cache.popitem(last=False)  # Remove least recently used

            return result

        wrapper.cache_clear = cache.clear
        return wrapper

    return decorating_function


@lru_cache_by_key(key_func=lambda prompt, llm, chat, embedding: embedding, maxsize=128)
def _cached_llm_inference(prompt, llm, chat, embedding):
    """Cached LLM inference: returns output for a given quantized embedding."""
    print("Computing inference...")
    print("Embedding:", embedding)
    return llm.process_prompt(prompt, chat=chat)



llm_response = handle_prompt("What is the capital of France?", chat = True)
print(llm_response.llm_output)




yo key:  (137, 159, 0, 120, 190, 171, 107, 165, 136, 111, 94, 118, 207, 71, 148, 118, 84, 156, 161, 167, 119, 121, 128, 125, 255, 151, 171, 143, 84, 132, 112, 137, 115, 138, 195, 88, 131, 144, 181, 180, 161, 108, 132, 163, 199, 177, 99, 163, 169, 135, 114, 153, 107, 132, 150, 144, 171, 198, 196, 128, 205, 145, 141, 189, 169, 134, 123, 238, 159, 100, 142, 85, 185, 137, 132, 128, 143, 124, 107, 119, 141, 124, 174, 115, 170, 209, 135, 159, 135, 153, 134, 162, 175, 140, 134, 109, 142, 159, 152, 136, 139, 139, 103, 88, 156, 169, 108, 70, 98, 73, 104, 154, 180, 141, 130, 135, 207, 81, 158, 140, 89, 142, 163, 120, 90, 149, 112, 159, 114, 145, 118, 122, 124, 118, 157, 171, 90, 81, 128, 179, 142, 176, 216, 127, 93, 52, 181, 99, 150, 179, 191, 154, 125, 125, 172, 104, 111, 160, 188, 129, 171, 123, 142, 127, 139, 85, 154, 171, 135, 168, 84, 102, 99, 113, 176, 93, 152, 150, 129, 105, 128, 104, 139, 172, 120, 154, 110, 149, 111, 111, 96, 179, 154, 81, 92, 104, 147, 133, 167, 155, 100, 153, 138, 149

In [4]:
cache = OrderedDict()
cache["kid"] = "shot"
if "kid" in cache:
    print("here")

here
